# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kelibia,36.8476,11.0939,26.56,65,0,3.84,TN,1721506018
1,1,adamstown,-25.0660,-130.1015,15.80,86,100,9.14,PN,1721505740
2,2,puerto natales,-51.7236,-72.4875,7.25,61,40,12.86,CL,1721505749
3,3,east london,-33.0153,27.9116,14.64,68,94,6.01,ZA,1721506022
4,4,dudinka,69.4058,86.1778,6.30,69,0,6.18,RU,1721506024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)


# Display the map
humidity_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp'] < 20) & (city_data_df['Max Temp'] > 10) & 
(city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
clean_city_df = ideal_city_df.dropna()

# Display sample data
clean_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,keflavik,64.0049,-22.5624,11.24,82,0,3.60,IS,1721506075
65,65,taikang,46.8333,124.4167,19.95,95,0,3.69,CN,1721506108
217,217,arauco,-37.2463,-73.3175,11.66,58,0,2.31,CL,1721506324
225,225,saint-pierre,-21.3393,55.4781,19.76,77,0,2.06,RE,1721506336
228,228,yemva,62.5900,50.8594,12.39,63,0,2.65,RU,1721506340


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
40,keflavik,IS,64.0049,-22.5624,82,
65,taikang,CN,46.8333,124.4167,95,
217,arauco,CL,-37.2463,-73.3175,58,
225,saint-pierre,RE,-21.3393,55.4781,77,
228,yemva,RU,62.5900,50.8594,63,
238,victoria falls,ZW,-17.9235,25.8430,27,
258,nampula,MZ,-15.1165,39.2666,77,
263,illapel,CL,-31.6308,-71.1653,74,
283,ringkobing,DK,56.0901,8.2440,77,
305,tolanaro,MG,-25.0319,46.9987,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "apiKey":geoapify_key,
    "limit": 5
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
keflavik - nearest hotel: Núpan Deluxe
taikang - nearest hotel: No hotel found
arauco - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
yemva - nearest hotel: Гостиница Континент
victoria falls - nearest hotel: Shearwater Explorers Village
nampula - nearest hotel: Hotel Lurio
illapel - nearest hotel: Domingo Ortiz de Rosas
ringkobing - nearest hotel: Town Living
tolanaro - nearest hotel: Hôtel Mahavokey
luau - nearest hotel: No hotel found
tromso - nearest hotel: Clarion Collection Hotel Aurora
gaiman - nearest hotel: yr hen ffordd
vennesla - nearest hotel: No hotel found
ribeirao branco - nearest hotel: 91
ambovombe - nearest hotel: No hotel found
leirvik - nearest hotel: Almaas Hotell Stord
sambava - nearest hotel: Royal Sava Hôtel


,City,Country,Lat,Lng,Humidity,Hotel Name
40,keflavik,IS,64.0049,-22.5624,82,Núpan Deluxe
65,taikang,CN,46.8333,124.4167,95,No hotel found
217,arauco,CL,-37.2463,-73.3175,58,No hotel found
225,saint-pierre,RE,-21.3393,55.4781,77,Tropic Hotel
228,yemva,RU,62.5900,50.8594,63,Гостиница Континент
238,victoria falls,ZW,-17.9235,25.8430,27,Shearwater Explorers Village
258,nampula,MZ,-15.1165,39.2666,77,Hotel Lurio
263,illapel,CL,-31.6308,-71.1653,74,Domingo Ortiz de Rosas
283,ringkobing,DK,56.0901,8.2440,77,Town Living
305,tolanaro,MG,-25.0319,46.9987,94,Hôtel Mahavokey


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].astype(str)

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Country", "Hotel Name"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)